Read in radar parameters and split the data into a 70-30 split for training and testing data.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error


In [2]:
radar_params = pd.read_csv('radar_parameters.csv', usecols=lambda x: x != 'Unnamed: 0')

radar_params

,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr)
0,23.144878,0.418637,-41.757733,0.005395,0.000290,0.000012,2.393520
1,22.737156,0.322850,-43.772069,0.005194,0.000360,0.000012,3.502699
2,26.869826,0.330948,-43.577399,0.013385,0.000903,0.000030,8.627561
3,28.540561,0.399480,-42.139731,0.018872,0.001036,0.000043,8.424447
4,30.500127,0.543758,-39.763087,0.027438,0.001157,0.000064,8.189291
...,...,...,...,...,...,...,...
18964,31.515997,0.579955,-39.244229,0.034048,0.001417,0.000080,10.648020
18965,29.993334,0.567935,-39.399188,0.024134,0.001032,0.000057,7.981875
18966,31.685913,0.655681,-38.375696,0.033971,0.001165,0.000081,6.822691
18967,32.980096,0.768586,-37.166218,0.043117,0.001285,0.000105,6.801169


In [3]:
radar_params.columns
radar_params.shape

(18969, 7)

In [4]:
# Arrange features and target (R, rain rate)

X_feat = radar_params.drop('R (mm/hr)', axis=1)
print(X_feat.shape)

Y_targ = radar_params['R (mm/hr)']
print(Y_targ.shape)


(18969, 6)
(18969,)


In [5]:
# Create 70-30 split
X_train, X_test, y_train, y_test = train_test_split(
    X_feat, Y_targ, test_size=0.30, random_state=42
)

Using the split created in (1), train a multiple linear regression dataset using the training dataset, and validate it using the testing dataset.  Compare the $R^2$ and root mean square errors of model on the training and testing sets to a baseline prediction of rain rate using the formula $Z = 200 R^{1.6}$.

In [ ]:
# Function to compute and print metrics
def report(name, y_true_train, y_pred_train, y_true_test, y_pred_test):
    r2_train = r2_score(y_true_train, y_pred_train)
    r2_test = r2_score(y_true_test, y_pred_test)
    rmse_train = np.sqrt(mean_squared_error(y_true_train, y_pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_true_test, y_pred_test))
    print(f"----- {name} -----")
    print(f"R^2 (train): {r2_train:.4f}   R^2 (test): {r2_test:.4f}")
    print(f"RMSE (train): {rmse_train:.4f}   RMSE (test): {rmse_test:.4f}")
    print()

# Compute baseline prediction using Zh only and Z = 200 R^{1.6} 
# Convert dBZ -> Z (linear)
def baseline_predict_from_Zh(Zh_vals):
    Z_linear = 10 ** (Zh_vals / 10.0)
    R_calc = (Z_linear / 200.0) ** (1.0 / 1.6)
    return R_calc

# Compute baseline on train and test
Zh_all = radar_params['Zh (dBZ)'].values

# Apply function to the Zh columns of train/test dataframes
baseline_train_pred = baseline_predict_from_Zh(X_train.iloc[:, 0])
baseline_test_pred = baseline_predict_from_Zh(X_test.iloc[:, 0])



In [18]:
# Report baseline metrics
report("Baseline (Z=200 R^1.6 -> predict R from Zh)", y_train, baseline_train_pred, y_test, baseline_test_pred)

# Multiple Linear Regression on all features
lr = LinearRegression(fit_intercept=True)
lr.fit(X_train, y_train)

y_train_pred_lr = lr.predict(X_train)
y_test_pred_lr = lr.predict(X_test)

report("Multiple Linear Regression (6 features)", y_train, y_train_pred_lr, y_test, y_test_pred_lr)

print(lr.coef_)
print(lr.intercept_)


----- Baseline (Z=200 R^1.6 -> predict R from Zh) -----
R^2 (train): 0.2756   R^2 (test): 0.3566
RMSE (train): 7.1440   RMSE (test): 7.1893

----- Multiple Linear Regression (6 features) -----
R^2 (train): 0.9879   R^2 (test): 0.9891
RMSE (train): 0.9229   RMSE (test): 0.9358

[ 1.59901626e-01  2.02690952e+00 -6.18193443e-01 -7.10460369e+01
  7.77892389e+03 -6.12208071e+03]
-29.22109494538222


Repeat 1 doing a grid search over polynomial orders, using a grid search over orders 0-9, and use cross-validation of 7 folds.  For the best polynomial model in terms of $R^2$, does it outperform the baseline and the linear regression model in terms of $R^2$ and root mean square error?

In [ ]:
# Grid search over polynomial orders 0-9 with 7-fold cross-validation
pipe = Pipeline([
    ("poly", PolynomialFeatures(degree=1, include_bias=True, interaction_only=False)),
    ("scaler", StandardScaler()),
    ("lr", LinearRegression())
])

param_grid = {
    "poly__degree": list(range(0, 10)),  
    "poly__include_bias": [True],        
    "poly__interaction_only": [False]
}

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=7,
    scoring="r2",
    n_jobs=-1,
    refit=True,
    verbose=1
)


In [17]:
grid.fit(X_train, y_train)


Fitting 7 folds for each of 10 candidates, totalling 70 fits


,estimator,Pipeline(step...egression())])
,param_grid,"{'poly__degree': [0, 1, ...], 'poly__include_bias': [True], 'poly__interaction_only': [False]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,7
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,degree,2


In [18]:
print("Best params:", grid.best_params_)
print(f"Best CV R^2 (training folds): {grid.best_score_:.4f}")
print()

Best params: {'poly__degree': 2, 'poly__include_bias': True, 'poly__interaction_only': False}
Best CV R^2 (training folds): 0.9970



In [19]:
# Evaluate best model on train/test
best_model = grid.best_estimator_
y_train_pred_poly = best_model.predict(X_train)
y_test_pred_poly = best_model.predict(X_test)

report(f"Best Polynomial degree={grid.best_params_['poly__degree']}", y_train, y_train_pred_poly, y_test, y_test_pred_poly)

----- Best Polynomial degree=2 -----
R^2 (train): 0.9996   R^2 (test): 0.9996
RMSE (train): 0.1672   RMSE (test): 0.1836



Repeat 1 with a Random Forest Regressor, and perform a grid_search on the following parameters. Can you beat the baseline, or the linear regression, or best polynomial model with the best optimized Random Forest Regressor in terms of $R^2$ and root mean square error?

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import time

param_grid = {
"bootstrap": [True, False],
"max_depth": [10, 100],
"max_features": ["sqrt", 1.0],
"min_samples_leaf": [1, 4],
"min_samples_split": [2, 10],
"n_estimators": [200, 1000]
}

# Create base random forest regressor and setup grid search 
rf = RandomForestRegressor(random_state=42)

grid_rf = GridSearchCV(
estimator=rf,
param_grid=param_grid,
cv=7,
scoring="r2",
n_jobs=-1,
verbose=2,
refit=True
)


In [ ]:
#t0 = time.time()
grid_rf.fit(X_train, y_train)
#t1 = time.time()
#print(f"Random Forest grid search done in {t1 - t0:.1f} s")
print("Best RF params:", grid_rf.best_params_)
print(f"Best CV R^2 (training folds): {grid_rf.best_score_:.4f}")
print()

Fitting 7 folds for each of 64 candidates, totalling 448 fits
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.7s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total

/home/faa6/envs/xarray-climate/lib/python3.14/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=  35.0s
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=  34.8s
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=  34.9s
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   5.5s
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   5.3s
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   5.3s
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   5.2s
[CV] END bootstrap=True, max_depth

Random Forest grid search done in 3759.6 s
Best RF params: {'bootstrap': True, 'max_depth': 100, 'max_features': 1.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best CV R^2 (training folds): 0.9814

In [28]:
# Evaluate best RF on train and test sets
best_rf = grid_rf.best_estimator_

y_train_pred_rf = best_rf.predict(X_train)
y_test_pred_rf = best_rf.predict(X_test)

In [ ]:
# Compute metrics (R^2 and RMSE)
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

r2_train_rf = r2_score(y_train, y_train_pred_rf)
r2_test_rf = r2_score(y_test, y_test_pred_rf)
rmse_train_rf = rmse(y_train, y_train_pred_rf)
rmse_test_rf = rmse(y_test, y_test_pred_rf)

print("----- Random Forest (best) -----")
print(f"R^2 (train): {r2_train_rf:.4f} R^2 (test): {r2_test_rf:.4f}")
print(f"RMSE (train): {rmse_train_rf:.4f} RMSE (test): {rmse_test_rf:.4f}")
print()



----- Random Forest (best) -----
R^2 (train): 0.9974 R^2 (test): 0.9879
RMSE (train): 0.4252 RMSE (test): 0.9858



In [ ]:
# Compare all 4 methods
models_summary = {
"Baseline": {
"r2_train": r2_score(y_train, baseline_train_pred),
"r2_test": r2_score(y_test, baseline_test_pred),
"rmse_train": rmse(y_train, baseline_train_pred),
"rmse_test": rmse(y_test, baseline_test_pred)
},
"LinearRegression": {
"r2_train": r2_score(y_train, y_train_pred_lr),
"r2_test": r2_score(y_test, y_test_pred_lr),
"rmse_train": rmse(y_train, y_train_pred_lr),
"rmse_test": rmse(y_test, y_test_pred_lr)
},
"BestPolynomial": {
"r2_train": r2_score(y_train, y_train_pred_poly),
"r2_test": r2_score(y_test, y_test_pred_poly),
"rmse_train": rmse(y_train, y_train_pred_poly),
"rmse_test": rmse(y_test, y_test_pred_poly)
},
"RandomForest": {
"r2_train": r2_train_rf,
"r2_test": r2_test_rf,
"rmse_train": rmse_train_rf,
"rmse_test": rmse_test_rf
}
}

print("Model comparison (higher R^2 better; lower RMSE better):")
print("Model\t\tR2_train\tR2_test\t\tRMSE_train\tRMSE_test")

for name, stats in models_summary.items():
    print(f"{name:15s}\t{stats['r2_train']:.4f}\t\t{stats['r2_test']:.4f}\t\t{stats['rmse_train']:.4f}\t\t{stats['rmse_test']:.4f}")


Model comparison (higher R^2 better; lower RMSE better):
Model		R2_train	R2_test		RMSE_train	RMSE_test
Baseline       	0.2756		0.3566		7.1440		7.1893
LinearRegression	0.9879		0.9891		0.9229		0.9358
BestPolynomial 	0.9996		0.9996		0.1672		0.1836
RandomForest   	0.9974		0.9879		0.4252		0.9858
